## The Goal: Test length of time needed to find significant differences in coupling index between CESM1 and CESM2. 
<br>
<b>Author:</b> Meg Fowler <br>
<b>Date:</b> 9 Sep 2020 <br><br>

In [1]:
# Import libraries 
# Import libraries 
import comet as cm 
import numpy as np 
import xarray as xr 
import pickle
import pandas as pd
import datetime 

# Plotting utils 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.util


## Read in data for CESM1 and CESM2 simulations 

In [2]:
#cesm1dir   = '/Users/meganfowler/Documents/NCAR/Analysis/Coupling_initial/data/CESM1/'
cesm1dir = '/glade/work/mdfowler/data/CESM1/'
cesm2dir = '/Users/meganfowler/Documents/NCAR/Analysis/Coupling_initial/data/day/'

# CESM2 file name structures
middleName_cesm2 = '_day_CESM2_amip_r10i1p1f1_gn_'
varNames_cesm2   = ['hfls','hfss','mrso','mrsos','tas','hurs','ps']
timeName_flx2    = ['19700101-19791231','19800101-19891231', 
                    '19900101-19991231','20000101-20091231']
timeName_soil2   = ['19500101-19991231','20000101-20150101']

# CESM1 file name structures 
endCLM_cesm1   = '_soilliqOnly.nc'
endIce_cesm1   = '_soiliceOnly.nc'
endFLX_cesm1   = '_sfcFluxes.nc'
timeName_cesm1 = ['1979-1989', '1990-1999', '2000-2006']


In [ ]:
# Read in test file to get lat/lon (which is same between sims)
fileName = cesm2dir+varNames_cesm2[0]+middleName_cesm2+timeName_flx2[0]+'.nc'
testDF   = xr.open_dataset(fileName, decode_times=True)

# Get lat and lon 
lat = testDF.lat.values
lon = testDF.lon.values 


### Read in CESM1 data

In [ ]:
# Read in files and get time as usable format 
file1 = cesm1dir+startCAM+timeName[0]+endFLX 
flxDF = xr.open_dataset(file1, decode_times=True)
flxDF['time'] = flxDF.indexes['time'].to_datetimeindex()
print('File 1 finished reading in...')

file2  = cesm1dir+startCAM+timeName[1]+endFLX 
flxDF2 = xr.open_dataset(file2, decode_times=True) 
flxDF2['time'] = flxDF2.indexes['time'].to_datetimeindex()
print('File 2 finished reading in...')

file3  = cesm1dir+startCAM+timeName[2]+endFLX 
flxDF3 = xr.open_dataset(file3, decode_times=True) 
flxDF3['time'] = flxDF3.indexes['time'].to_datetimeindex()
print('File 3 finished reading in...')

# Concat in one array 
flux_full  = xr.concat([flxDF,  flxDF2, flxDF3], dim="time")


In [ ]:
# Read in files and get time as usable format 
file1 = cesm1dir+startCLM+timeName[0]+endCLM 
soilDF = xr.open_dataset(file1, decode_times=True)
soilDF['time'] = soilDF.indexes['time'].to_datetimeindex()
print('File 1 finished reading in...')
soilDF = soilDF.where(soilDF.levgrnd<2.12193400e-01, drop=True)  # Ignore lower down levels 

file2  = cesm1dir+startCLM+timeName[1]+endCLM 
soilDF2 = xr.open_dataset(file2, decode_times=True) 
soilDF2['time'] = soilDF2.indexes['time'].to_datetimeindex()
print('File 2 finished reading in...')
soilDF2 = soilDF2.where(soilDF2.levgrnd<2.12193400e-01, drop=True)

file3  = cesm1dir+startCLM+timeName[2]+endCLM 
soilDF3 = xr.open_dataset(file3, decode_times=True) 
soilDF3['time'] = soilDF3.indexes['time'].to_datetimeindex()
print('File 3 finished reading in...')
soilDF3 = soilDF3.where(soilDF3.levgrnd<2.12193400e-01, drop=True)


# Concat in one array 
soil_full  = xr.concat([soilDF,  soilDF2, soilDF3], dim="time")


In [ ]:
# Read in files and get time as usable format 
file1 = cesm1dir+startCLM+timeName[0]+endIce 
soilDF = xr.open_dataset(file1, decode_times=True)
soilDF['time'] = soilDF.indexes['time'].to_datetimeindex()
print('File 1 finished reading in...')
soilDF = soilDF.where(soilDF.levgrnd<2.12193400e-01, drop=True)  # Ignore lower down levels 

file2  = cesm1dir+startCLM+timeName[1]+endIce 
soilDF2 = xr.open_dataset(file2, decode_times=True) 
soilDF2['time'] = soilDF2.indexes['time'].to_datetimeindex()
print('File 2 finished reading in...')
soilDF2 = soilDF2.where(soilDF2.levgrnd<2.12193400e-01, drop=True)

file3  = cesm1dir+startCLM+timeName[2]+endIce 
soilDF3 = xr.open_dataset(file3, decode_times=True) 
soilDF3['time'] = soilDF3.indexes['time'].to_datetimeindex()
print('File 3 finished reading in...')
soilDF3 = soilDF3.where(soilDF3.levgrnd<2.12193400e-01, drop=True)

# Concat in one array 
soilIce_full  = xr.concat([soilDF,  soilDF2, soilDF3], dim="time")


In [ ]:
# Load pressure level of LCL (computed separately in Get_LCLpressure.py and saved as pickle file)
lclPath = '/glade/work/mdfowler/scripts/Coupling_CAM6CLM5/processed_data/LCL-pressure-HeightAboveGround_CESM1.p'
Plcl    = pickle.load( open( lclPath, "rb" ) )


In [ ]:
# To be able to easily access years, months, days - use Pandas 
dates    = pd.DatetimeIndex(soil_full['time'].values)


In [ ]:
# Read in land fraction 
filePath = '/glade/work/mdfowler/data/CESM1/cam5.1_amip_1d_002.clm2.h1.1979-01-01-00000.nc'
exDF = xr.open_dataset(filePath,decode_times=False)

landMask              = exDF.landmask.values.astype(float)
landMask[landMask==0] = np.nan

landArea = exDF.area.values * exDF.landfrac.values

**Get 10 cm soil moisture by itself**

In [ ]:
# Sum over top 4 levels to get top 11cm of soil water/ice 
soilLiq_11cm = np.nansum(soil_full.SOILLIQ.values,    axis=1)*landMask
soilIce_11cm = np.nansum(soilIce_full.SOILICE.values, axis=1)*landMask

# Get as close to "mrsos" as possible by including liq and ice, and dividing by land area 
soil_11cm = (soilLiq_11cm + soilIce_11cm)


### Read in CESM2 data

**Get same time period for CESM2 data as in CESM1**

## Compute coupling index for CESM1 and CESM2 in select period

## Plot coupling index for both simulations - are there big differences between the two?